## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key= g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,69.12,83,20,23.02,few clouds
1,1,Castro,BR,-24.7911,-50.0119,70.88,81,14,11.12,few clouds
2,2,Hobart,AU,-42.8794,147.3294,61.27,81,75,10.36,light rain
3,3,Rantepao,ID,-2.9701,119.8978,66.90,97,100,1.28,overcast clouds
4,4,Alice Springs,AU,-23.7000,133.8833,58.55,77,0,5.75,clear sky


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]

In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,69.12,few clouds,-33.9258,18.4232,
2,Hobart,AU,61.27,light rain,-42.8794,147.3294,
3,Rantepao,ID,66.90,overcast clouds,-2.9701,119.8978,
12,New Norfolk,AU,60.93,overcast clouds,-42.7826,147.0587,
14,Cidreira,BR,69.94,overcast clouds,-30.1811,-50.2056,
16,Rincon,US,66.06,scattered clouds,32.2960,-81.2354,
17,Busselton,AU,63.05,clear sky,-33.6500,115.3333,
20,Hermanus,ZA,66.79,light rain,-34.4187,19.2345,
30,Luwingu,ZM,61.68,overcast clouds,-10.2621,29.9271,
31,Rocha,UY,66.20,clear sky,-34.4833,-54.3333,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,69.12,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
2,Hobart,AU,61.27,light rain,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
3,Rantepao,ID,66.90,overcast clouds,-2.9701,119.8978,Hotel Luta Resort
12,New Norfolk,AU,60.93,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
14,Cidreira,BR,69.94,overcast clouds,-30.1811,-50.2056,Hotel Castelo
16,Rincon,US,66.06,scattered clouds,32.2960,-81.2354,Days Inn Rincon
17,Busselton,AU,63.05,clear sky,-33.6500,115.3333,Observatory Guest House
20,Hermanus,ZA,66.79,light rain,-34.4187,19.2345,Misty Waves Boutique Hotel
30,Luwingu,ZM,61.68,overcast clouds,-10.2621,29.9271,"MR.C SIMON CHABALA (Maje, Home Place)"
31,Rocha,UY,66.20,clear sky,-34.4833,-54.3333,Hotel Arrarte


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))